In [ ]:
# Import needed things
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from solsys_code.views import JPLSBId

In [ ]:
# Declare a time and point of interest (start of commissioning on the Extended Chandra Deep Field South)
obs_time = Time('2024-11-11 09:00:00', scale='utc')
ecdfs = SkyCoord(53.13, -28.10, unit=u.deg, frame='icrs')

In [ ]:
# Declare a JPLSBId object and initialize with the ComCam field of view (converted to a half-width)
rubin_comcam = JPLSBId(fov_ra_hwidth=40 * u.arcmin / 2.0, fov_dec_hwidth=40 * u.arcmin / 2.0)
print(f'MPC Code: {rubin_comcam.mpc_code} FOV= {rubin_comcam.fov_ra_hwidth:.3f} x {rubin_comcam.fov_dec_hwidth:.3f}')

# Query the JPL Small Body ID tool (can take 30-60s). The defaults (`raw_response`=True, verbose=True) will show details of the query and return the raw (JSON) response


In [ ]:
results = rubin_comcam.query_center(obs_time, ecdfs)
print(results.keys())

In [ ]:
# Looking at the summary (the RA and Dec are '-'-separated and minus signs in the Dec are convered to `M`)
for key, value in results['summary'].items():
    print(f'{key:<14}: {value}')

In [ ]:
# Per object Data is in 'data_first_pass' with the fields as described in 'fields_first'
print('\n'.join(results['fields_first']))
print()
print(results['data_first_pass'][0])

In [ ]:
# If instead, we set `raw_response` to False, the list of objects will be turned into a QTable with the position as a SkyCoord
# and Quantity columns
table = rubin_comcam.query_center(obs_time, ecdfs, raw_response=False)
print(table)
print(table.colnames)